<a href="https://colab.research.google.com/github/PRAVEEN-CS33/OwlUp/blob/main/yolo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
import os
#google drive mounting
drive.mount('/content/drive')
os.chdir('/content/drive/MyDrive/Colab Notebooks/yolo')
print(os.getcwd())

Mounted at /content/drive
/content/drive/MyDrive/Colab Notebooks/yolo


In [ ]:
!pip install ultralytics
!pip install supervision
!pip install geopy
!pip install twilio
# !git clone https://github.com/ultralytics/ultralytics
# !pip install roboflow

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 586.5/586.5 kB 22.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 205.1/205.1 kB 26.2 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.1/42.1 kB 5.8 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 42.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 62.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.5/114.5 kB 15.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 29.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt

# Read the input image
img = cv2.imread('/content/IMG20220406002301.jpg')

# Convert the image to grayscale
gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

# Apply a false color map
colormap = cv2.COLORMAP_JET
thermal = cv2.applyColorMap(gray, colormap)
cv2.imwrite('/content/gray_image.jpg', thermal)
# Display the output image
# plt.imshow('Thermal Image', thermal)
# cv2.waitKey(0)
# cv2.destroyAllWindows()

In [ ]:
!unzip '/content/drive/MyDrive/Copy of anomaly_image.zip' -d '/content/drive/MyDrive/Colab Notebooks/yolo/dataset'

In [ ]:
from roboflow import Roboflow
#dataset download from Roboflow
rf = Roboflow(api_key="KQSYBMVR3uU9mYfiP6ER")
project = rf.workspace("saad-salman").project("gun_dataset_6")
dataset = project.version(5).download("yolov8")

In [ ]:
from ultralytics import YOLO
#loading, training, validate the model 
model = YOLO('yolov8l.pt')
model.train(data='/content/drive/MyDrive/Colab Notebooks/yolo/dataset/anomaly_image/data.yaml', epochs=5 , imgsz=640)
model.val()

In [ ]:
from ultralytics import YOLO
#load the best model & prediction
model = YOLO('/content/drive/MyDrive/Colab Notebooks/yolo/runs/detect/train6/weights/best.pt')
model.predict('/content/drive/MyDrive/Colab Notebooks/yolo/dataset/anomaly/train/images/3_mp4-63_jpg.rf.63e9f3baa13de2c847bd5ba3ca4048a6.jpg', save=True, imgsz=320, conf=0.1)

In [ ]:
import smtplib
from email.mime.multipart import MIMEMultipart
from email.mime.image import MIMEImage
from email.mime.text import MIMEText
from email.mime.base import MIMEBase
from email import encoders
from geopy.geocoders import Nominatim
from datetime import datetime
from twilio.rest import Client
import geopy
import time
import csv


def loc_time():

  geolocator = geopy.Nominatim(user_agent="my_app", timeout=10)

  location = geolocator.geocode("Coimbatore")

  # locname = geolocator.reverse(f"{location.latitude}, {location.longitude}")
  latitude = location.latitude
  longitude = location.longitude

  address = " WXQ5+58F, BK Pudur, Sugunapuram East, Kuniyamuthur, Tamil Nadu 641008"

  now = datetime.now()
  date = now.date()
  time = now.time()

  return ([latitude,longitude,address,date,time])
# print(loc_time()[3])
# print(loc_time()[4])

def alert_mail(Incident):

  msg = MIMEMultipart()
  msg['From'] = '727721eucs107@skcet.ac.in'
  msg['To'] = '727721eucs107@skcet.ac.in'
  msg['Subject'] = '⚠Crime Alert'

  # message body
  # Latitude: {latitude}\nLongitude: {longitude}\nAddress: {address}\nDate: {date}\nTime: {time}

  msg.attach(MIMEText(f"Details of the detected crime :\n\nCamera_Id : 1\nIncident  : {Incident}\nLatitude  : {loc_time()[0]}\nLongitude : {loc_time()[1]}\nAddress   : {loc_time()[2]}\nDate      : {loc_time()[3]}\nTime      : {loc_time()[4]}\nLevel     : High"))

  # Attach the image
  # with open(img, 'rb') as f:
  #   img = MIMEImage(f.read())
  #   msg.attach(img)
    
  video_file = '/content/drive/MyDrive/Colab Notebooks/evidence.mp4'

  with open(video_file, 'rb') as f:
       video_data = f.read()

  # # Attach the video file to the email
  part = MIMEBase('application', 'octet-stream')
  part.set_payload(video_data)
  encoders.encode_base64(part)
  part.add_header('Content-Disposition', f'attachment; filename="{os.path.basename(video_file)}"')
  msg.attach(part)

  # Connect to SMTP server and send email
  smtp_server = 'smtp.gmail.com'
  smtp_port = 587

  smtp_username = '727721eucs107@skcet.ac.in'
  smtp_password = 'tskpraveen@skcet33'

  smtp_conn = smtplib.SMTP(smtp_server, smtp_port)
  smtp_conn.starttls()
  smtp_conn.login(smtp_username, smtp_password)
  smtp_conn.sendmail(msg['From'], msg['To'], msg.as_string())
  smtp_conn.quit()
  print("Alert sent")
  
print(1)
def call():

  account_sid = 'AC3f19fc87f8c5d0d3f6d25e2a05fe08b4'
  auth_token = '077c6666955a4ca319623c312c6fafd6'
  client = Client(account_sid, auth_token)

  call = client.calls.create(
                        twiml='<Response><Say voice="alice">  Hello, this is a high risk alert for this area . and Details of this alert have been sent via email. Thank youu </Say></Response>',
                        to='+91 8098167783',
                        from_='+91 8098167783'
                    )
  message = client.messages.create(
                        to='+91 8098167783', 
                        from_='+14407501802', 
                        body="Hello, this is a high-risk alert from WXQ5+58F, BK Pudur, Sugunapuram East, Kuniyamuthur, Tamil Nadu 641008. More Details of this alert have been sent via email. Thank you."
                    )
  print(call.sid)
# call()
# def database():

#   id = 1;
#   # data = ['Camera_Id', 'Incident', 'Latitude','Longitude', 'Address', 'Date', 'Time', 'Level']
#   data = [id,'Theft', '11.0018115', '76.9628425',' WXQ5+58F, BK Pudur, Sugunapuram East, Kuniyamuthur, Tamil Nadu 641008',' 2023-03-28', '07:33:35.012395', 'High']
#   with open('/content/drive/MyDrive/Colab Notebooks/yolo/database/data.csv', 'a') as f:
#     writer = csv.writer(f)
#     # row = ','.join(str(x) for x in data) + '\n'
#     writer.writerow(data)
#     id +=1

# res = {
#     'Camera_Id' : 1, 
#     'Incident': "Theft",
#     'Latitude': loc_time()[0] ,
#     'Longitude': loc_time()[1],
#     'Address': loc_time()[2],
#     'Date': f"{loc_time()[3]}", 
#     'Time': f"{loc_time()[4]}",
#     'Level':"High"
# }
# print(res)
# # alert_mail("t")
# # call()
# # loc_time()

1


In [ ]:
from ultralytics import YOLO
import supervision as sv
import numpy as np
import cv2

def main():

  cap = cv2.VideoCapture('/content/drive/MyDrive/Colab Notebooks/yolo/dataset/CCTV Footage/1.mp4')
  # cap.set(cv2.CAP_PROP_FRAME_WIDTH, 1280)
  # cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 720)

  model = YOLO('/content/drive/MyDrive/Colab Notebooks/yolo/runs/detect/train6/weights/best.pt')

  while True:
     red, frame = cap.read()

     result = model(frame, agnostic_nms=True)[0]
     detection = sv.Detections.from_yolov8(result)

     class_ids = [0,1,2,3,4,5,6,7,8]
     mask = np.in1d(detection.class_id, class_ids)
     detection = detection[mask]
     
     result = any(x == y for x in detection.class_id for y in class_ids)
     if (result):

      #  cv2.imwrite('/content/drive/MyDrive/Colab Notebooks/yolo/pred/frame1.jpg', frame)

       temp_cap = cap

       current_frame = int(cap.get(cv2.CAP_PROP_POS_FRAMES))

       fps = temp_cap.get(cv2.CAP_PROP_FPS)

       start_frame = max(current_frame - 300, 0)
       end_frame = current_frame + 300

       out_path = '/content/drive/MyDrive/Colab Notebooks/evidence.mp4'
       fourcc = cv2.VideoWriter_fourcc(*'mp4v')

       out = cv2.VideoWriter(out_path, fourcc, fps, (int(temp_cap.get(3)), int(temp_cap.get(4))))

       temp_cap.set(cv2.CAP_PROP_POS_FRAMES, start_frame)

       while cap.isOpened():
           ret, frame = temp_cap.read()
           if ret:
               out.write(frame)

               if cap.get(cv2.CAP_PROP_POS_FRAMES) == end_frame:
                   break
           else:
               break
       

       temp_cap.release()
       out.release()

       call()
       alert_mail(str(detection.class_id))
      #  database()
       
     if not red:
       break
     if cv2.waitKey(25) & 0xFF == ord('q'):
       break
     
    #  cap.release()

if __name__ == '__main__':
  main()


0: 480x640 1 gun, 52.0ms
Speed: 1.8ms preprocess, 52.0ms inference, 1.5ms postprocess per image at shape (1, 3, 640, 640)


CA96b7ba325776756947183a04850ce4e5


WARNING ⚠️ 'source' is missing. Using 'source=https://ultralytics.com/images/bus.jpg'.

Found https://ultralytics.com/images/bus.jpg locally at bus.jpg


Alert sent


image 1/1 /content/drive/MyDrive/Colab Notebooks/yolo/bus.jpg: 640x480 (no detections), 98.2ms
Speed: 3.0ms preprocess, 98.2ms inference, 0.8ms postprocess per image at shape (1, 3, 640, 640)


In [ ]:
res = {
    'Camera_Id' : 1, 
    'Incident': "Theft",
    'Latitude': loc_time()[0] ,
    'Longitude': loc_time()[1],
    'Address': loc_time()[2],
    'Date': f"{loc_time()[3]}", 
    'Time': f"{loc_time()[4]}",
    'Level':"High"
}
print(res)

In [ ]:
!pip install pyngrok==4.1.1
!pip install flask_ngrok
!ngrok authtoken "2OCQLU3la21DNSDu4kvJIWAbTum_6TCGp3z9Ys1ajDk7UdF2t" 
!pip install flask-cors
!pip install flask-socketio

In [ ]:
from flask_ngrok import run_with_ngrok
from flask import Flask, request, jsonify
from flask_cors import CORS, cross_origin
from flask_socketio import SocketIO

app = Flask(__name__)
socketio = SocketIO(app, cors_allowed_origins='*')
run_with_ngrok(app)
CORS(app)

@socketio.on('event_name')
def handle_event(data):
    # Process the data and emit a response
    response_data = {'message': 'Response from server'}
    socketio.emit('response_event', response_data)

@app.route('/', methods=['POST', 'GET'])
@cross_origin(origins=['http://localhost:3000', 'https://example.com'], headers=['Content-Type', 'Authorization'])
def predict():
    response = jsonify({'data': 'Hello, World!'})
    socketio.emit('response_event', {'message': 'Hello from server'})
    return response
  
if __name__ == '__main__':
    app.run()



# @app.route('/', methods=['POST', 'GET'])
# def predict():
#     response = jsonify({'data': 'Hello, World!'})
#     return response

# @socketio.on('event_name')
# def handle_event(data):
#     response_data = {'message': 'Hello from server'}
#     emit('response_event', response_data)


 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit


 * Running on http://0681-35-237-241-136.ngrok-free.app
 * Traffic stats available on http://127.0.0.1:4040


### **Audio classifier**

**Install dependencies**

In [ ]:
!pip install tensorflow tensorflow-gpu tensorflow-io matplotlib

load de

### **check**

In [ ]:
import folium
from geopy.geocoders import Nominatim

# create a geolocator object
geolocator = Nominatim(user_agent="my_app")

# use the geolocator to get the current location based on IP address
location = geolocator.geocode("")

# create a folium map centered on the current location
m = folium.Map(location=[location.latitude, location.longitude], zoom_start=5)

# add a marker for the current location
folium.Marker([location.latitude, location.longitude]).add_to(m)

# display the map
m

In [ ]:
import cv2

cap = cv2.VideoCapture('/content/drive/MyDrive/Colab Notebooks/yolo/Screenrecording_20230327_120501 (online-video-cutter.com).mp4')

temp_cap = cap

current_frame = int(cap.get(cv2.CAP_PROP_POS_FRAMES))

fps = temp_cap.get(cv2.CAP_PROP_FPS)

start_frame = max(current_frame - 900, 0)
end_frame = current_frame + 900

out_path = '/content/drive/MyDrive/Colab Notebooks/yolo/pred/clip.mp4'
fourcc = cv2.VideoWriter_fourcc(*'mp4v')

out = cv2.VideoWriter(out_path, fourcc, fps, (int(temp_cap.get(3)), int(temp_cap.get(4))))

temp_cap.set(cv2.CAP_PROP_POS_FRAMES, start_frame)

while cap.isOpened():
    ret, frame = temp_cap.read()
    if ret:
        out.write(frame)

        if cap.get(cv2.CAP_PROP_POS_FRAMES) == end_frame:
            break
    else:
        break

temp_cap.release()
out.release()
cv2.destroyAllWindows()


In [ ]:
import io
import base64
from IPython.display import HTML
import ipywidgets as widgets
import cv2

# Load the video file
video_path = '/content/drive/MyDrive/Colab Notebooks/yolo/pred/output.mp4'
# video_path = cv2.VideoCapture(0)
video_file = io.open(video_path, 'r+b').read()

# Encode the video as base64
video_url = base64.b64encode(video_file).decode()

# Create the HTML video player
html = """
<video width="480" height="360" controls>
  <source src="data:video/mp4;base64,{0}" type="video/mp4">
</video>
""".format(video_url)

# Display the video player
HTML(html)


In [ ]:
import geopy
import time

geolocator = geopy.Nominatim(user_agent="my_app", timeout=10)

def get_location(address):
    try:
        location = geolocator.geocode(address)
        return location.latitude, location.longitude
    except geopy.exc.GeocoderUnavailable:
        time.sleep(5)
        return get_location(address)
get_location('Coimbatore')


(11.0018115, 76.9628425)

### **CLI**

In [ ]:
from ultralytics import YOLO

In [ ]:
!yolo detect train data='/content/drive/MyDrive/Colab Notebooks/yolo/gun_dataset_6-5/data.yaml' model=yolov8n.pt epochs=100 imgsz=640

In [ ]:
!yolo detect val model='/content/drive/MyDrive/Colab Notebooks/yolo/runs/detect/train/weights/best.pt'

In [ ]:
!yolo detect predict model='/content/drive/MyDrive/Colab Notebooks/yolo/runs/detect/train/weights/best.pt' source='/content/drive/MyDrive/Colab Notebooks/yolo/gun_dataset_6-5/train/images/11-015_jpg.rf.303a9d03e06481203201291e61d3f8cd.jpg'

In [ ]:
model = YOLO('/content/drive/MyDrive/Colab Notebooks/yolo/runs/detect/train/weights/best.pt')
model.predict(source='/content/drive/MyDrive/Colab Notebooks/yolo/gun-131126.mp4', save = True)